In [10]:
import random

def generateBoard(cols, rows, mines):
    board = [[0 for x in range(cols)] for y in range(rows)]

    #generate mine locations
    minePos = random.sample(range(rows * cols), mines)
    for pos in minePos:
        row, col = divmod(pos, cols)
        board[row][col] = 'M'

    #count neighbors
    for row in range(rows):
        for col in range(cols):
            if board[row][col] == 'M':
                continue
            count = 0
            for rowShift in [-1, 0, 1]:
                for colShift in [-1, 0, 1]:
                    r= row + rowShift
                    c = col + colShift
                    if (0 <= r < rows) and (0 <= c < cols) and (board[r][c] == 'M'):
                        count += 1
            board[row][col] = count

    return board

def generatePlayArea(cols, rows):
    PlayArea = [['X' for x in range(cols)] for y in range(rows)]
    return PlayArea

def checkSpot(board, col, row):
    result = board[row][col]
    return result

In [7]:
board = generateBoard(30,16,99)
playArea = generatePlayArea(30,16)

for row in board:
    print(row)
print("----------")
for row in playArea:
    print(row)

[0, 1, 2, 'M', 2, 'M', 'M', 'M', 2, 'M', 'M', 1, 0, 0, 1, 1, 2, 1, 1, 0, 0, 0, 1, 1, 1, 1, 'M', 'M', 'M', 1]
[0, 1, 'M', 4, 5, 4, 4, 2, 2, 3, 3, 2, 1, 1, 2, 'M', 3, 'M', 1, 1, 1, 2, 2, 'M', 1, 1, 2, 3, 2, 1]
[0, 1, 2, 'M', 'M', 'M', 2, 0, 1, 2, 'M', 2, 2, 'M', 2, 2, 'M', 2, 1, 1, 'M', 2, 'M', 3, 2, 0, 0, 0, 0, 0]
[1, 1, 3, 3, 5, 'M', 2, 0, 1, 'M', 3, 4, 'M', 3, 1, 1, 1, 2, 1, 2, 1, 3, 3, 'M', 1, 1, 2, 2, 2, 1]
[1, 'M', 3, 'M', 3, 1, 1, 0, 1, 1, 2, 'M', 'M', 2, 1, 1, 1, 1, 'M', 1, 0, 1, 'M', 2, 2, 2, 'M', 'M', 2, 'M']
[2, 2, 3, 'M', 2, 0, 0, 0, 0, 0, 2, 4, 4, 2, 1, 'M', 1, 1, 1, 1, 0, 1, 1, 1, 2, 'M', 4, 2, 2, 1]
['M', 2, 2, 3, 2, 1, 0, 0, 0, 0, 1, 'M', 'M', 1, 1, 2, 2, 1, 0, 1, 2, 2, 1, 1, 3, 'M', 2, 0, 0, 0]
[1, 2, 'M', 3, 'M', 1, 0, 0, 0, 0, 1, 2, 2, 1, 0, 1, 'M', 1, 0, 1, 'M', 'M', 1, 1, 'M', 2, 2, 1, 1, 0]
[0, 1, 2, 'M', 2, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 2, 2, 1, 1, 2, 3, 2, 1, 1, 1, 2, 2, 'M', 1, 0]
[0, 0, 1, 2, 2, 2, 1, 3, 'M', 3, 1, 2, 'M', 1, 1, 'M', 1, 0, 2, 'M', 2, 0, 0, 0, 0,

In [11]:
import copy
import numpy as np

def countNeighborUnknowns(playArea, col, row):
    unseenCount = 0
    flagCount = 0
    for colShift in [-1, 0, 1]:
        for rowShift in [-1, 0, 1]:
            curCol = col + colShift
            curRow = row + rowShift

            if (curRow >= 0 and curRow < len(playArea) and curCol >= 0 and curCol < len(playArea[0])):
                if(playArea[curRow][curCol] == 'X'):
                    unseenCount += 1
                if(playArea[curRow][curCol] == 'F'):
                    flagCount += 1

    return unseenCount, flagCount



def getFrontier(playArea):
    frontier = []
    for i in range(len(playArea)):
        for j in range(len(playArea[0])):
            unseenCount, flagCount = countNeighborUnknowns(playArea, j, i)
            if (unseenCount >= 1):

                #could append both unseen count or flag count here?
                frontier.append((j,i, unseenCount, flagCount))

    return frontier

def depthFirstSearch(board, playArea, col, row):
    if (row < 0 or row >= len(board) or col < 0 or col >= len(board[0]) or playArea[row][col] != 'X'):
        return

    playArea[row][col] = checkSpot(board, col, row)

    if playArea[row][col] == 'M':
        print("lost during dfs")
        return

    # if no nearby mines, check all nearby squares
    if playArea[row][col] == 0:
        for i in [-1, 0, 1]:
            for j in [-1, 0, 1]:
                depthFirstSearch(board, playArea, col+i, row+i)
    elif isinstance(playArea[row][col], int):
        # if it is a number, we need to backtrack?
        return

def solveCertainMoves(board, playArea):
    frontier = getFrontier(playArea)
    for (col, row, unseenCount, flagCount) in frontier:
        if (playArea[row][col] - flagCount == unseenCount):
            for colShift in [-1, 0, 1]:
                for rowShift in [-1, 0, 1]:
                    curCol = col + colShift
                    curRow = row + rowShift

                    if (curRow >= 0 and curRow < len(playArea) and curCol >= 0 and curCol < len(playArea[0]) and playArea[curRow][curCol] == 'X'):
                        playArea[curRow][curCol] = 'F'
        elif (playArea[row][col] - flagCount == 0 and unseenCount > 0):
            # if all mines have been seen, but there are still more neighbor squares to explore
            for colShift in [-1, 0, 1]:
                for rowShift in [-1, 0, 1]:
                    curCol = col + colShift
                    curRow = row + rowShift

                    if (curRow >= 0 and curRow < len(playArea) and curCol >= 0 and curCol < len(playArea[0]) and playArea[curRow][curCol] == 'X'):
                        depthFirstSearch(board, playArea, curCol, curRow)

def guess(board, playArea):
    frontier = getFrontier(playArea)
    bestOdd = 1.0
    bestCellRow = -1
    bestCellCol = -1

    #iterate across frontier, getting cell with best odds and guess that one of it's adjacent neighbors is not a mine





def solve(board, playArea):
    pastState = copy.deepcopy(playArea)

    #this needs to be changed to "until complete"
    for row in range(len(board)):
        for col in range(len(board[0])):
            cellVal = checkSpot(board, col, row)

            if cellVal == 'M':
                print("lost during solve")
                continue
            
            #explore zeros
            depthFirstSearch(board, playArea, col, row)

            #create frontier
            #solve certain moves
            solveCertainMoves(board, playArea)

            #check if past state is same as cur state
            # if so, we need to guess
            np_playArea = np.array(playArea)
            np_pastState = np.array(playArea)
            if (np.all(np_playArea == np_pastState)):
                guess(board, playArea)

            pastState = copy.deepcopy(playArea)



    

solve(board, playArea)

lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost during solve
lost durin